In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv(r"C:\Users\HP\Documents\Python Scripts\energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
df=df.drop(['date','lights'],axis=1)

In [4]:
#SCALING THE DATASET USING MinMax SCALER
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df)

#to convert scaled dataset into a pandas dataframe
df = pd.DataFrame(scaled_df,columns=df.columns)

X=df.drop('Appliances',axis=1)
y=df['Appliances']


#to divide datset into train and testing data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

#import the linear model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

#to predict the test set
predicted_values = linear_model.predict(X_test)

In [5]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,predicted_values)
round(mae,3)

0.05

In [6]:
rss=np.sum(np.square(y_test-predicted_values))
round(rss,2)

45.35

In [7]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test,predicted_values)
round(r2_score,2)

0.15

In [8]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

In [9]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [10]:
get_weights_df(linear_model,X,'Linear_weight_coefficients')

,Features,Linear_weight_coefficients
0,rv1,-2.759603e+10
1,RH_2,-4.567063e-01
2,T_out,-3.218632e-01
3,T2,-2.361857e-01
4,T9,-1.899368e-01
5,RH_8,-1.575968e-01
6,RH_out,-7.767082e-02
7,RH_7,-4.461318e-02
8,RH_9,-3.980205e-02
9,T5,-1.565491e-02


In [11]:
#Using Ridge regreession
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.4)
ridge_model.fit(X_train,y_train)

Ridge(alpha=0.4)

In [12]:
ridge_predictions = ridge_model.predict(X_test)

In [13]:
rmse=np.sqrt(mean_squared_error(y_test,ridge_predictions))
round(rmse,3)

0.088

In [14]:
#Using Ridge regreession
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X_train,y_train)

Lasso(alpha=0.001)

In [15]:
len(lasso_model.coef_[lasso_model.coef_ !=0])

4

In [16]:
predicted_values = lasso_model.predict(X_test)
predicted_values

array([0.07370267, 0.08143458, 0.07716072, ..., 0.07792848, 0.09034412,
       0.08359255])

In [17]:
rmse=np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.094